In [ ]:
import numpy as np
from gtda.plotting import plot_point_cloud
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from gtda.plotting import plot_diagram
from sklearn.ensemble import RandomForestClassifier

Helper function to make some point clouds X:


In [ ]:
def make_point_clouds(n_samples_per_shape: int, n_points: int, noise: float):
    #--- circles ---
    circle_point_clouds = [
        np.asarray(
            [
                [np.sin(t) + noise * (np.random.rand(1)[0] - 0.5), np.cos(t) + noise * (np.random.rand(1)[0] - 0.5), 0]
                for t in range((n_points ** 2))
            ]
        )
        for kk in range(n_samples_per_shape)
    ]
    circle_labels = np.zeros(n_samples_per_shape)  # label circles with 0

    #--- spheres ---
    sphere_point_clouds = [
        np.asarray(
            [
                [
                    np.cos(s) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.cos(s) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.sin(s) + noise * (np.random.rand(1)[0] - 0.5),
                ]
                for t in range(n_points)
                for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]
    sphere_labels = np.ones(n_samples_per_shape)     # label spheres with 1

    #--- tori ---
    torus_point_clouds = [
        np.asarray(
            [
                [
                    (2 + np.cos(s)) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                    (2 + np.cos(s)) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.sin(s) + noise * (np.random.rand(1)[0] - 0.5),
                ]
                for t in range(n_points)
                for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]

    torus_labels = 2 * np.ones(n_samples_per_shape)     # label tori with 2

    #--- plane ---
    coefs = list(zip((1+6*np.random.rand(n_samples_per_shape)).astype(int), (1+6*np.random.rand(n_samples_per_shape)).astype(int)))
    plane_point_clouds = [
        np.asarray(
            [
                [
                    x,
                    y,
                    coef[0]*x + coef[1]*y,
                ]
                for x in np.linspace(-1, 1, n_points)
                for y in np.linspace(-1, 1, n_points)
            ]
        )
        for coef in coefs
    ]
    plane_labels = 3 * np.ones(n_samples_per_shape) # label plane with 3

    point_clouds = np.concatenate((circle_point_clouds, sphere_point_clouds, torus_point_clouds, plane_point_clouds))
    labels = np.concatenate((circle_labels, sphere_labels, torus_labels, plane_labels))

    return point_clouds, labels

In [ ]:
n_samples_per_class = 10
point_clouds, labels = make_point_clouds(n_samples_per_class, n_points = 10, noise=0.1)
print(point_clouds.shape)
print(labels.shape)

In [ ]:
plot_point_cloud(point_clouds[0]).show()

In [ ]:
plot_point_cloud(point_clouds[11]).show()

In [ ]:
plot_point_cloud(point_clouds[20]).show()

In [ ]:
plot_point_cloud(point_clouds[-1]).show()

In [ ]:
# @title
# Track connected components, loops, and voids
homology_dimensions = [0, 1, 2]

# Collapse edges to speed up H2 persistence calculation!
persistence = VietorisRipsPersistence(
    metric = "euclidean",
    homology_dimensions=homology_dimensions,
    n_jobs=6,
    collapse_edges=False
)

diagrams = persistence.fit_transform(point_clouds)

In [ ]:
plot_diagram(diagrams[0]).show() # Circle

In [ ]:
plot_diagram(diagrams[11]).show()  # Sphere

In [ ]:
plot_diagram(diagrams[20]).show() # Torus

In [ ]:
plot_diagram(diagrams[-3]).show() # Plane

In [ ]:
persistence_entropy = PersistenceEntropy()

# calculate topological feature matrix
X = persistence_entropy.fit_transform(diagrams)
plot_point_cloud(X).show()

In [ ]:
rf = RandomForestClassifier(oob_score=True)
rf.fit(X, labels)
print(f"OOB score: {rf.oob_score:.3f}")